정확도 낮음  
제대로 학습이 되지 않음  
학습을 지속할 수록 감소 쪽으로 더 몰리거나 왔다가다 하는 경향
stride 커지면 관찰 기간 더 길어지면서 감소 쪽으로 더 몰리는 것도 같음
dilation 도 그럴 것 같지만 특별히 더 몰리지는 않음

## Load data

In [774]:
import copy as cp

import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt

In [3]:
import os, itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import sys
#lib_s = '/home/adminuser/public/libs/pytorch_examples/basic_networks'
lib_s = '/home/bwlee/work/codes/examples/basic_networks'
sys.path.append(lib_s)
import generate_model as nets

## Make input data
interest.csv has only closed price.   
On the other hand, zipline needs OLHCV(?) format for each index  
make dummy columns and put 'close' value to them  
make csv file for each index  

In [763]:
f_s0 = 'interest.csv'
df0 = pd.read_csv(f_s0, index_col='date')
df0 = df0.iloc[::-1] # 과거에서 현재로 정렬순서 변경

### Add more data

In [764]:
# normalized input, norm val, tgt_date, tgt_val, tgt_diff_ratio, 
class Environ():
    def __init__(self):
        self.indices = ['trea3', 'trea5', 'trea10']
        #self.tgt_index = 'trea10'
        self.tgt_index = 'trea3'
        # input window size
        self.input_size = 15
        # prediction is made after algo.pred_period from the present
        self.pred_period = 5

env = Environ() # setting values

In [765]:
#dff0 = df0[:200] # test
dff0 = df0
dff = dff0

In [766]:
diff_periods = [int(env.input_size/4*x) for x in [1, 2, 3]]
for diff_period in diff_periods:
    diff = dff0.diff(periods=diff_period)
    ii = diff_period
    columns = ['d%d_diff_trea3'%(ii), 'd%d_diff_trea5'%(ii), 'd%d_diff_trea10'%(ii)]
    diff.columns = columns
    dff = dff.merge(diff, how='inner', on='date')
dff['diff35'] = dff['trea3'] - dff['trea5']
dff['diff310'] = dff['trea3'] - dff['trea10']
dff['diff510'] = dff['trea5'] - dff['trea10']

dff = dff.dropna()

## prepare data without time confusion
Seperate two actions

In [767]:
def push_data(env, data, mkt_data):
    """
    read past data and push them to mkt_data
    :param data: dataframe info from csv file
    :param mkt_data: dataframe to be used for learning
    """
    for ii_days in range(len(data)):
        if ii_days+1 < env.input_size:
            continue
        elif ii_days+1 == env.input_size:
            env.start_date = data.iloc[ii_days].name
        
        predicting_date = data.iloc[ii_days].name
        # historical data from today to env.input_size behind days
        history = data.iloc[ii_days+1-env.input_size:ii_days+1]
        
        ## normalize
        """
        #  : look back 1 year
        lookback_step = 250
        lookback_step = env.input_size
        if ii_days+1 < lookback_step:
            cumul = data.iloc[:ii_days+1]
        else:
            cumul = data.iloc[ii_days+1-lookback_step:ii_days+1]
        
        mean_val = cumul.mean() 
        range_val = 2*cumul.std()
        """
        
        mean_val = history.mean() 
        range_val = history.max() - history.min()
        
        norm_history = (history-mean_val)/range_val
        mkt_data[predicting_date] = [norm_history, mean_val, range_val, 
                                     None, None, None, None, None]
        
            
def push_tgt(env, data, mkt_data):
    """
    read future data and push them to mkt_data
    future_prediction_date is a future date when model want to predict
    past_predicting_date is a past date when model made a prediction with given information
    :param data: dataframe info from csv file
    :param mkt_data: dataframe to be used for learning
    """
    for ii_days in range(len(data)):
        ii_predicting = ii_days - env.pred_period
        predicting_date = data.iloc[ii_predicting].name
        if ii_predicting < 0 or predicting_date < env.start_date:
            continue
        
        past_history = mkt_data[predicting_date][0]
        mean_val = mkt_data[predicting_date][1]
        range_val = mkt_data[predicting_date][2]
        
        past_price = (past_history.iloc[-1]*range_val+mean_val)[env.tgt_index] # 49.6
        
        #past_price = (past_history.iloc[-5]*range_val+mean_val)[env.tgt_index] # 56.3
        
        #mkt_data[predicting_date][0] = past_history[:-4]
        
        #past_price = (past_history.iloc[-3]*range_val+mean_val)[env.tgt_index] # 53.0, # 53.0
        #past_price = (past_history.iloc[-13]*range_val+mean_val)[env.tgt_index] # 67.4
        #past_price = (past_history*range_val+mean_val).mean()[env.tgt_index] # 58.4
        #past_price = (past_history*range_val+mean_val)[:-3].mean()[env.tgt_index] # 63.2
        #past_price = (past_history*range_val+mean_val)[:-5].mean()[env.tgt_index] # 63.7
        
        #past_price = (past_history.iloc[-10]*range_val+mean_val)[env.tgt_index] # 66.4

        #past_price = (past_history.iloc[-5:].mean()*range_val+mean_val)[env.tgt_index] # 48.1
        #past_price = (past_history.iloc[-3:].mean()*range_val+mean_val)[env.tgt_index] # 48.1
        
        #past_price = (past_history.iloc[-10:].mean()*range_val+mean_val)[env.tgt_index] # 55.9
        #past_price = (past_history.iloc[-10:-5].mean()*range_val+mean_val)[env.tgt_index] # 60.4
        
        
        prediction_date = data.iloc[ii_days].name
        tgt_price = data.iloc[ii_days][env.tgt_index]
        #tgt_price = (data.iloc[ii_days-1:ii_days+2].mean())[env.tgt_index]
        
        tgt_ratio = (tgt_price - past_price)/past_price*10
        
        mkt_data[predicting_date][3] = prediction_date
        mkt_data[predicting_date][4] = np.single(tgt_price)
        mkt_data[predicting_date][5] = np.single(tgt_ratio)
        
        mkt_data[predicting_date][6] = np.single(past_price)
        mkt_data[predicting_date][7] = predicting_date

### Push past data

In [769]:
mkt_data = {} # object to store data for learning
push_data(env, dff, mkt_data)

### Check data are pushed correctly
* no dates should be ahead of key date
* prediction info are not pushed yet

In [896]:
print(list(mkt_data.keys())[:3])

['2000-11-18', '2000-11-20', '2000-11-21']


In [897]:
mkt_data[list(mkt_data.keys())[0]]

[               trea3     trea5    trea10  d3_diff_trea3  d3_diff_trea5  \
 date                                                                     
 2000-11-02  0.455471  0.457619  0.468205       0.156444       0.139912   
 2000-11-03  0.486005  0.493333  0.498974       0.129778       0.126754   
 2000-11-04  0.486005  0.493333  0.498974       0.169778       0.172807   
 2000-11-06  0.516539  0.529048  0.537436       0.176444       0.205702   
 2000-11-07  0.516539  0.536190  0.537436       0.149778       0.179386   
 2000-11-08  0.402036  0.407619  0.414359       0.049778       0.060965   
 2000-11-09 -0.040712 -0.070952 -0.062564      -0.363556      -0.412719   
 2000-11-10 -0.391858 -0.363810 -0.324103      -0.670222      -0.689035   
 2000-11-11 -0.391858 -0.363810 -0.324103      -0.570222      -0.570614   
 2000-11-13 -0.483461 -0.456667 -0.462564      -0.263556      -0.215351   
 2000-11-14 -0.452926 -0.463810 -0.462564       0.069778       0.047807   
 2000-11-15 -0.315522 -0.

In [209]:
env.start_date

'2000-11-06'

### Push future data

In [770]:
push_tgt(env, dff, mkt_data)
mkt_data = {key: val for key, val in mkt_data.items() if (val[-1] is not None)}

### CHECK tgt push is correctly made
end value of '2000-11-23' * norm == prediction value 

In [226]:
print(mkt_data['2000-11-11'])
print(mkt_data['2000-11-23'])

[               trea3     trea5    trea10
date                                    
2000-10-26  0.300000  0.272099  0.293963
2000-10-27  0.105970  0.131358  0.152231
2000-10-28  0.105970  0.131358  0.152231
2000-10-30  0.091045  0.123951  0.152231
2000-10-31  0.150746  0.175802  0.183727
2000-11-01  0.105970  0.123951  0.136483
2000-11-02  0.128358  0.123951  0.104987
2000-11-03  0.158209  0.160988  0.136483
2000-11-04  0.158209  0.160988  0.136483
2000-11-06  0.188060  0.198025  0.175853
2000-11-07  0.188060  0.205432  0.175853
2000-11-08  0.076119  0.072099  0.049869
2000-11-09 -0.356716 -0.424198 -0.438320
2000-11-10 -0.700000 -0.727901 -0.706037
2000-11-11 -0.700000 -0.727901 -0.706037, trea3     7.664000
trea5     7.866333
trea10    8.143333
dtype: float64, trea3     0.670
trea5     0.675
trea10    0.635
dtype: float64, '2000-12-05', 7.07, -1.0108074]
[               trea3     trea5    trea10
date                                    
2000-11-07  0.681934  0.687143  0.711795
2000-11-

In [1]:
## Training
### Data preparation

## Data preparation for Conv1d
In RNN, inputs are given as (time, indices)  
In Conv1d, indices are given as channel, which need to (channel: indices, features: time)  
(batch, channel=indices, data_size=seq_size)

In [791]:
class Index_data(Dataset):
    def __init__(self, in_data0, dates, addi=None):
        """
        :param in_data: dict of (df_past_normed, norm_val, prediction_date, tgt_price, tgt_ratio)
        :param addi: (optional) additional non-sequential elements that can be added to in_data
        """
        in_data = cp.deepcopy(in_data0)
        if addi is not None:
            assert list(in_data.keys()) == list(addi.keys())
            
        # change dataframe to float numpy array
        for key, val in in_data.items():
            # transform in_data into Tensor
            if not 'Tensor' in str(type(in_data[key][0])) :
                #print('shape', (in_data[key][0].to_numpy()).shape, (in_data[key][1].to_numpy()).shape)
                in_data[key][0] = torch.tensor(in_data[key][0].to_numpy(np.single))
                in_data[key][1] = torch.tensor(in_data[key][1].to_numpy(np.single))
                in_data[key][2] = torch.tensor(in_data[key][2].to_numpy(np.single))
            
            if addi is not None:
                in_data[key].append(addi[key])
            else:
                # None is not accept in Dataloader, so give null value
                in_data[key].append('NULL')
            
        self.in_data = in_data
        self.i2dates = {}
        for ii, date1 in enumerate(dates):
             self.i2dates[ii] = date1    
        
    def __getitem__(self, ii_date):
        """
        :return: (df_past_normed, norm_val, prediction_date, tgt_price, tgt_ratio)
        """
        predicting_date = self.i2dates[ii_date]
        return self.in_data[predicting_date]
    
    def __len__(self):
        return len(self.in_data)
    
    def transpose(self):
        for key, val in self.in_data.items():
            self.in_data[key][0] = torch.transpose(val[0], 0, 1)
            self.in_data[key][1] = val[1].view(-1,1)
            self.in_data[key][2] = val[2].view(-1,1)

In [772]:
#erase data that do not have prediction target
traintest_dates = sorted(list(mkt_data.keys()))
n_train = len(traintest_dates)*8//10
n_test = len(traintest_dates) - n_train
train_dates = traintest_dates[:n_train]
test_dates = traintest_dates[n_train:]

train_data = {}
test_data = {}
for key, val in mkt_data.items():
    if key in train_dates:
        train_data[key] = val
    elif key in test_dates:
        test_data[key] = val

In [252]:
test_data['2021-01-15']

[            trea3  trea5    trea10
 date                              
 2020-12-24 -0.296  0.180 -0.375758
 2020-12-28 -0.196 -0.145 -0.648485
 2020-12-29  0.264  0.230 -0.284848
 2020-12-30  0.064  0.430  0.169697
 2020-12-31  0.064  0.430  0.169697
 2021-01-04 -0.236  0.180  0.351515
 2021-01-05 -0.596 -0.570 -0.321212
 2021-01-06 -0.136  0.105  0.351515
 2021-01-07  0.064  0.105  0.133333
 2021-01-08  0.204  0.080  0.133333
 2021-01-11 -0.036 -0.570 -0.503030
 2021-01-12  0.204 -0.170  0.078788
 2021-01-13  0.164 -0.270  0.078788
 2021-01-14  0.404  0.180  0.351515
 2021-01-15  0.064 -0.195  0.315152,
 trea3     0.966800
 trea5     1.317800
 trea10    1.712667
 dtype: float64,
 trea3     0.050
 trea5     0.040
 trea10    0.055
 dtype: float64,
 '2021-01-22',
 0.99,
 0.20618556,
 0.97,
 '2021-01-15']

In [792]:
## this is an example, I already add differences
def get_addi(in_data):
    #print(in_data)
    ret = {}
    for key, val in in_data.items():
        if not 'Tensor' in str(type(in_data[key][0])) :
            history_val = torch.tensor(val[0][['trea3','trea5', 'trea10']].to_numpy(np.single))
        else:
            history_val = val[0][:,:3] # only use three elements : ['trea3','trea5', 'trea10']
        addi1 = history_val[-1] - history_val[-5]
        addi2 = history_val[-1] - history_val[-7]
        addi3 = history_val[-1] - history_val[-10]
        addi4 = history_val[-1] - history_val[-3]
        addi5 = history_val[-3] - history_val[-7]
        addi6 = history_val[-3] - history_val[-10]
        addi7 = history_val[-5] - history_val[-7]
        addi8 = history_val[-5] - history_val[-10]
        addi = torch.cat((addi1, addi2, addi3, addi4, addi5, addi6, addi7, addi8), dim=-1)    
        ret[key] = addi
    return ret

In [793]:
# CNN needs transposed data
batch_size = 64
""" # no additional
train_dset = Index_data(train_data, train_dates)
test_dset = Index_data(test_data, test_dates)
"""
train_dset = Index_data(train_data, train_dates, get_addi(train_data))
test_dset = Index_data(test_data, test_dates, get_addi(test_data))
train_dset.transpose()
test_dset.transpose()
train_loader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dset, batch_size=batch_size, shuffle=False)

In [116]:
for ii in range(len(train_dset)):
    if type(train_dset[ii][-1]) is not torch.Tensor:
        print(train_dset[ii])

In [832]:
class Predictor(nets.Net):
    """
    1D CNN is constructed.
    You can constructed with 2D CNN
    """
    def __init__(self, net=None, downnet=None, dropout=0.35,
                 loss=None, optimizer=None, device='cuda:2',
                 dim_hiddens=[100,200,300,50], len_seq = 15, size_kernels=[3,4,5], 
                 hidden_out=None, down_hiddens=[1, 1], add_down=-1,
                 dilations=None, pads=None, strides=None):
        """
        net is consist of [embed, net, downnet]
        dilation will be treated in another class
        initial seq_width, which corresponds to kernel_size, is 15(time window size)
        :param downnet: define downstream job
        :down_hiddens: multiply out_dimension of convolution
        :dim_hiddens: size of features
        """
        assert len(dim_hiddens)-1 == len(size_kernels)
        if dilations is None:
            dilations = [1 for _ in range(len(dim_hiddens))]
        if pads is None:
            pads = [0 for _ in range(len(dim_hiddens))]
        if strides is None:
            strides = [1 for _ in range(len(dim_hiddens))]
        
        super(Predictor, self).__init__(loss=loss, device=device)
        
        if net is None:
            self.net = self._set_cnns(dim_hiddens, size_kernels, strides, pads, 
                                      dilations, dropout, len_seq)
        else:
            assert hidden_out is not None
            self.hidden_out = dim_out
            self.net = net
            
        if downnet is None:
            down_dims = [self.hidden_out*x for x in down_hiddens] + [1]
            if add_down>0:
                down_dims[0] += add_down
            print('down_dims', down_dims)
            self.downnet = nets.get_MLP(down_dims, dropout=0.35, end=True)
        else:
            self.downnet = downnet
        
        self.init_weights()
        
        self.net = self.net.to(device)
        self.downnet = self.downnet.to(device)
        
        parms = list(self.net.parameters())
        parms += list(self.downnet.parameters())
        self.optimizer = optimizer(parms)
     
    def _set_cnns(self, dim_hiddens, size_kernels, strides, pads, dilations,
                 dropout, len_seq):
        """
        in_channels, out_channels are dimension of features
        """
        convs = []
        bnorms = []
        activation = nn.ReLU()
        dropout = nn.Dropout(dropout)
        # Setting should be changed by padding, stride, dilation
        out_len_seq = len_seq
        for ii in range(1, len(dim_hiddens)):
            size_kernel = size_kernels[ii-1]
            in_size_feat = dim_hiddens[ii-1]
            out_size_feat = dim_hiddens[ii]
            stride = strides[ii-1]
            pad = pads[ii-1]
            dilation = dilations[ii-1]
            print('stride', stride, 'pad', pad, 'dilation', dilation, 'size_kernel', size_kernel)

            conv1 = nn.Conv1d(in_channels=in_size_feat, 
                              out_channels=out_size_feat,
                              kernel_size=size_kernel,
                              stride=stride,
                              padding=pad,
                              dilation=dilation)
            out_len_seq = int((out_len_seq + 2*pad - dilation*(size_kernel-1)-1)/stride) + 1
            print('out_len_seq', out_len_seq)
            convs.append(conv1)
            if out_len_seq < 1:
                raise Exception('Out dimension is less than 0, control parameters')
            bnorms.append(nn.BatchNorm1d(num_features=out_size_feat))
            
        last_n_channel = dim_hiddens[-1]
        # for data transforming to 1D
        self.hidden_out =  last_n_channel * out_len_seq
        
        conv_nets = []
        for conv1, bnorm in zip(convs, bnorms):
            conv_nets.append(dropout)
            conv_nets.append(conv1)
            conv_nets.append(activation)
            conv_nets.append(bnorm)
            
        return nn.Sequential(*conv_nets)
        
    def set_train(self):
        self.net.train()
        self.downnet.train()
        
    def set_eval(self):
        self.net.eval()
        self.downnet.eval()
    
    def init_weights(self):
        for ii, layer in enumerate(self.net):
            if 'Conv' in str(layer):
                torch.nn.init.xavier_uniform_(self.net[ii].weight)
        for ii, layer in enumerate(self.downnet):
            if 'Linear' in str(layer):
                torch.nn.init.xavier_uniform_(self.downnet[ii].weight)
                #torch.nn.init.xavier_normal_(self.downnet[ii].weight)

    def forward(self, x, addi=('NULL')):
        out = self.net(x)
        dim_batch = out.shape[0]
        # concatenate all data into 1D data
        # This transforms CNN filtered data to be used for downnet which is MLP
        out = out.reshape(dim_batch, -1) 
        if 'Tensor' in str(type(addi)):
            out = torch.cat((out, addi),dim=1)
        out = self.downnet(out) 
        return out.view(-1)

    def run_eval(self, data):
        self.set_eval()
        loss = 0
        outs = None
        tgts = None
        with torch.no_grad():
            for i_batch, data_batch in enumerate(data):
                data_in = data_batch[0]
                mean_val = data_batch[1]
                range_val = data_batch[2]
                pred_date = data_batch[3]
                tgt = data_batch[5]
                addi = data_batch[-1]
                
                data_in = data_in.to(self.device)
                tgt = tgt.to(self.device)
                if 'Tensor' in str(type(addi)):
                    addi = addi.to(self.device)
                    
                out = self.forward(data_in, addi)
                #print('loss', loss, self.loss)
                loss += self.loss(out, tgt).cpu().numpy()
                out = out.cpu().numpy()
                #print('loss===', loss)
                tgt = tgt.cpu().numpy()
                if outs is None:
                    outs = out
                    tgts = tgt
                else:
                    outs = np.concatenate((outs, out), axis=0)
                    tgts = np.concatenate((tgts, tgt), axis=0)
        loss /= 1.0*(i_batch+1)
        print('evaluate', 'loss', loss, 'accuracy : define function')
        return outs, tgts, loss
    
    def run_batch(self, i_batch, data_batch):
        self.optimizer.zero_grad()
        data_in = data_batch[0]
        mean_val = data_batch[1]
        range_val = data_batch[2]
        pred_date = data_batch[3]
        tgt = data_batch[5]
        addi = data_batch[-1]
        
        data_in = data_in.to(self.device)
        tgt = tgt.to(self.device)
        if 'Tensor' in str(type(addi)):
            addi = addi.to(self.device)
        out = self.forward(data_in, addi)
        
        loss = self.loss(out, tgt)
        loss.backward()
        self.optimizer.step()
        return loss.detach().cpu().item()

In [1072]:
len_seq = train_dset[0][0].shape[-1]
# Add difference variables
n_features = train_dset[0][0].shape[0]
down_hiddens = [1,2,2] # multiples of input size
size_kernels=[3,3,3,3]
dilations=[1,2,3]
dilations=None
dilations=[1,1,1,2]
#strides=[1,2,3]
#strides=[1,1,1]
strides=None
# actual feature size is dim_hidden * out_len_seq
#dim_hiddens = [n_features] + [10, 10, 20]
dim_hiddens = [n_features] + [n_features, n_features, n_features, n_features]
loss = nn.MSELoss() # which combines logsoftmax and nll loss
optimizer = optim.Adam
device = torch.device("cuda:2")

In [1073]:
predictor = Predictor(loss=loss, optimizer=optimizer, 
                      device=device, len_seq= len_seq,
                      dim_hiddens=dim_hiddens, down_hiddens=down_hiddens, 
                      size_kernels=size_kernels, add_down=3*8,
                      strides=strides,
                      dropout=0.1,
                      dilations=dilations,
                     )
                      #dilations=dilations)

stride 1 pad 0 dilation 1 size_kernel 3
out_len_seq 13
stride 1 pad 0 dilation 1 size_kernel 3
out_len_seq 11
stride 1 pad 0 dilation 1 size_kernel 3
out_len_seq 9
stride 1 pad 0 dilation 2 size_kernel 3
out_len_seq 5
down_dims [99, 150, 150, 1]


In [1106]:
predictor.run_train(n_epoch=10, data=train_loader)

epoch 0 loss 0.08031472558183457
eval_train evaluate loss 0.06857562576656911 accuracy : define function
epoch 1 loss 0.08039418674671828
eval_train evaluate loss 0.06909842639049488 accuracy : define function
epoch 2 loss 0.07980629162334685
eval_train evaluate loss 0.06832085939040825 accuracy : define function
epoch 3 loss 0.07947979939740095
eval_train evaluate loss 0.0658079196712864 accuracy : define function
epoch 4 loss 0.07956971283724058
eval_train evaluate loss 0.06639901220576087 accuracy : define function
epoch 5 loss 0.07974817420342076
eval_train evaluate loss 0.06665357141130006 accuracy : define function
epoch 6 loss 0.07915768622240024
eval_train evaluate loss 0.07384842567479433 accuracy : define function
epoch 7 loss 0.07898845345671497
eval_train evaluate loss 0.06544970415198981 accuracy : define function
epoch 8 loss 0.08075423160595681
eval_train evaluate loss 0.06615514658502679 accuracy : define function
epoch 9 loss 0.07853573277147848
eval_train evaluate los

In [1107]:
rets = {}
tgts = None
preds = None
predictor.set_eval()
with torch.no_grad():
    for data_batch in test_loader:
        data_in = data_batch[0]
        tgt = data_batch[5]
        
        data_in = data_in.to(device)
        addi = data_batch[-1]
        if 'Tensor' in str(type(addi)):
            addi = addi.to(device)
        tgt = tgt.numpy()
        #out = predictor.forward(data_in).cpu().numpy()
        out = predictor.forward(data_in, addi).cpu().numpy()
        if preds is None:
            preds = out
            tgts = tgt
        else:
            preds = np.concatenate((preds, out), axis=0)
            tgts = np.concatenate((tgts, tgt), axis=0)
            
        prediction_dates = data_batch[3]
        past_prices = data_batch[6]
        predicting_dates = data_batch[7]
        for ii, date1 in enumerate(prediction_dates):
            out1 = out[ii]
            out_orig = out1/10*past_prices[ii] + past_prices[ii]
            rets[date1] = {'predicting_date': predicting_dates[ii],
                          'pred': out_orig,
                           'past': past_prices[ii],
                           'tgt': data_batch[4][ii]}
            
preds = preds.reshape(-1)
tgts = tgts.reshape(-1)

In [1108]:
sum1 = np.sum((preds > 0) & (tgts > 0))
sum2 = np.sum((preds < 0) & (tgts < 0))
print(sum1, np.sum(tgts>0))
print(sum2, np.sum(tgts<0))
print(len(preds), sum1+sum2, 1.0*(sum1+sum2)/len(preds))

147 482
403 568
1068 550 0.5149812734082397


In [1109]:
limit = list(rets.keys()).index('2021-01-04')
sum1 = np.sum((preds[limit:] > 0) & (tgts[limit:] > 0))
sum2 = np.sum((preds[limit:] < 0) & (tgts[limit:] < 0))
print(sum1, np.sum(tgts[limit:]>0))
print(sum2, np.sum(tgts[limit:]<0))
print(len(preds[limit:]), sum1+sum2, 1.0*(sum1+sum2)/len(preds[limit:]))

17 44
19 24
71 36 0.5070422535211268


In [1022]:
list(preds)[:20]

[0.012098761,
 0.06398107,
 0.034371212,
 -0.014316494,
 -0.026728772,
 -0.008717187,
 0.00914385,
 0.0051481742,
 -0.015788577,
 -0.033726607,
 -0.02699127,
 -0.025021302,
 -0.017667959,
 -0.017090008,
 -0.025116883,
 -0.010800019,
 -0.0011157971,
 -0.025244031,
 -0.02163313,
 -0.0034951027]

* high dropout(0.35) is not good

In [22]:
rets_2021 = {key: val for key, val in rets.items() if key>'2021-01-01'}
rows = list(rets_2021['2021-01-04'].keys())
cols = list(rets_2021.keys())

In [27]:
rets_2021b = {}
for col in cols:
    for row in rows:
        rets_2021b.setdefault(row, {})
        val = rets_2021[col][row]
        if type(val) is not str:
            rets_2021b[row][col] = rets_2021[col][row].item()        
        else:
            rets_2021b[row][col] = rets_2021[col][row]

In [28]:
pd.DataFrame(rets_2021b).to_csv('./rets.csv', index=True)

In [21]:
rets_2021

{'2021-01-04': {'predicting_date': '2020-12-24',
  'pred': tensor(0.9607),
  'past': tensor(0.9403),
  'tgt': tensor(0.9550)},
 '2021-01-05': {'predicting_date': '2020-12-28',
  'pred': tensor(0.9542),
  'past': tensor(0.9453),
  'tgt': tensor(0.9370)},
 '2021-01-06': {'predicting_date': '2020-12-29',
  'pred': tensor(0.9781),
  'past': tensor(0.9630),
  'tgt': tensor(0.9600)},
 '2021-01-07': {'predicting_date': '2020-12-30',
  'pred': tensor(0.9640),
  'past': tensor(0.9690),
  'tgt': tensor(0.9700)},
 '2021-01-08': {'predicting_date': '2020-12-31',
  'pred': tensor(0.9614),
  'past': tensor(0.9733),
  'tgt': tensor(0.9770)},
 '2021-01-11': {'predicting_date': '2021-01-04',
  'pred': tensor(0.9480),
  'past': tensor(0.9650),
  'tgt': tensor(0.9650)},
 '2021-01-12': {'predicting_date': '2021-01-05',
  'pred': tensor(0.9365),
  'past': tensor(0.9540),
  'tgt': tensor(0.9770)},
 '2021-01-13': {'predicting_date': '2021-01-06',
  'pred': tensor(0.9564),
  'past': tensor(0.9507),
  'tgt': t